# Outil de cartographie: carte de chaleur (France) par région.



Il est nécessaire de télécharger les limites géographiques des départements de France disponibles sur le Github: c'est le dossier "france_regions". Il faut ensuite donner le chemin vers le fichier "nouvelles_regions.shp" contenu dans ce dossier.

In [ ]:

#@title Installation des bibliothèques nécessaires et connexion à un compte Google Drive.

''' 
Google Colab notebook.
Python == 3.7.11

BaOIA - La Contemporaine - Université de Nanterre
'''


## Installation des bibliothèques et connexion au compte Google Drive


!pip install pgeocode==0.3.0
!pip install geopandas==0.9.0
from google.colab import drive
import json
import pgeocode
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
%matplotlib inline

drive.mount('/content/drive/')
%cd /content/drive/My Drive/romans_scolaires/regions_map/

In [ ]:
#@markdown Le fichier d'entrée des données est un fichier .JSON contenant les lieux ainsi que les départements auxquels ils se rapportent
chemin_fichier_json = "/content/drive/My Drive/romans_scolaires/test_textblocks.json" #@param {type:"string"}

liste_codes = []

with open(chemin_fichier_json, "r") as f :
  contenu_json = json.load(f)
  for lieu in contenu_json:
    code = contenu_json[lieu]["Code Postal"]
    liste_codes.append(code)

nomi = pgeocode.Nominatim('fr')
df_regions = nomi.query_postal_code(liste_codes)

## Compter nombre par régions
df_nombre_regions=df_regions['state_name'].value_counts()
df_nombre_regions.to_excel('df_nombre_regions.xlsx', index = True, header=True)
df2_nombre_regions = pd.DataFrame(df_nombre_regions)

#@markdown Chemin vers le fichier "nouvelles_regions.shp":
chemin_fichier_shp = '/content/drive/My Drive/romans_scolaires/regions_map/nouvelles_regions/nouvelles_regions.shp' #@param {type:"string"}

regions = gpd.read_file(chemin_fichier_shp)
regions.to_excel('voir_regions.xlsx', index = True, header=True)

merged = regions.set_index('NOUVEAUNOM').join(df2.set_index('NOUVEAUNOM'))
merged = merged.reset_index()
merged = merged.fillna(0)
merged[['NOUVEAUNOM', 'code_supra', 'geometry', 'state_name']]


fig, ax = plt.subplots(1, figsize=(20, 10))
ax.axis('off')
color = 'Oranges'
vmin, vmax = 0, 535
sm = plt.cm.ScalarMappable(cmap=color, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)
cbar.ax.tick_params(labelsize=20)
cbar.ax.set_title("Nombre de lieux cités par région sur \n l'ensemble du corpus de romans scolaires.", fontdict={'fontsize': '13', 'fontweight' : '1'})
merged.plot('state_name', cmap=color, linewidth=0.8, ax=ax, edgecolor='0.8', figsize=(20,10))
plt.legend()
#@markdown Nom du fichier avec la carte (sortie) au format .png:
nom_carte_png = 'carte_regions.png' #@param {type:"string"}


plt.savefig(nom_carte_png)
